In [1]:
import matplotlib.pyplot as plt
import numpy as np 
from math import sqrt 
from math import floor
from math import ceil
from math import exp 
import pickle
from operator import itemgetter
import time
from datetime import timedelta
import matplotlib
from matplotlib import cm, colors
from numpy import amin, amax, ravel
import pickle


In [2]:
steps = [(-1, 0, 0), (0, -1, 0), (0,0,-1), (1, 0, 0), (0,0,1), (0,1,0)]

In [3]:
def get_all_saws(current_paths, length, current_conformation):
    """ Все созданные конформации лежат в первом аргументе функции get_all_saws, 
    поэтому первым аргументом нужно подавать переменную, которой ранее присвоили пустой список
    
    """
    if(length==1):
        current_paths.append(current_conformation)
    else:
        for step in steps:
            new_point = (current_conformation[-1][0]+step[0], current_conformation[-1][1]+step[1],current_conformation[-1][2]+step[2] )
            if new_point in current_conformation:
                continue
            else:
                temp_path = current_conformation.copy()
                temp_path.append(new_point)
                get_all_saws(current_paths, length-1,  temp_path)

In [4]:
def get_sequences(length):
    """Получаем все последовательности заданной длины, состоящие из 0 и 1.
    1 - это H
    0 - это Р"""
    if(length ==1):
        return [[-1], [1]]
    else:
        previous = get_sequences(length - 1)
        result = [] 
        for i in range(len(previous)):
            current = previous[i].copy()
            current.append(-1)
            result.append(current.copy())
            current[-1] = 1
            result.append(current)
            #print(result) 
        return sorted(result)

In [5]:
get_sequences(4)

[[-1, -1, -1, -1],
 [-1, -1, -1, 1],
 [-1, -1, 1, -1],
 [-1, -1, 1, 1],
 [-1, 1, -1, -1],
 [-1, 1, -1, 1],
 [-1, 1, 1, -1],
 [-1, 1, 1, 1],
 [1, -1, -1, -1],
 [1, -1, -1, 1],
 [1, -1, 1, -1],
 [1, -1, 1, 1],
 [1, 1, -1, -1],
 [1, 1, -1, 1],
 [1, 1, 1, -1],
 [1, 1, 1, 1]]

In [6]:
def end_to_end_distance(path):
    """Евклидова метрика"""
    distance = (path[-1][0]-path[0][0])**2+(path[-1][1]-path[0][1])**2+(path[-1][2]-path[0][2])**2
    return distance 

In [7]:
 
def Ising_energy(saw, seq):
    """Функция принимает на вход конформацию на плоскости(список координат). 
    Возвращает число топологических соседей для нее """
    k=0
    for i in range(1, len(saw)-1):
        
        for step in steps:
            new_point = (saw[i][0]+step[0], saw[i][1]+step[1], saw[i][2]+step[2] )
            if (new_point in saw    ):
                position = saw.index(new_point)
                k=k+seq[i]*seq[position]
    for step in steps:
        new_point_begin = (saw[0][0]+step[0], saw[0][1]+step[1],saw[0][2]+step[2]  )
        new_point_end= ( saw[-1][0]+step[0], saw[-1][1]+step[1], saw[-1][2]+step[2])
        
        if(new_point_begin in saw ):
            position = saw.index(new_point_begin)
            k=k+seq[0]*seq[position]
 
        if( new_point_end in saw    ):
            position = saw.index(new_point_end)
            k=k+seq[-1]*seq[position]
        
        
     
    return -k//2

In [8]:
conformat = []

start_conformation = [(0,0, 0)]
get_all_saws(conformat, 6,start_conformation)
print(len(conformat))

3534


In [9]:
conformat[8]

[(0, 0, 0), (-1, 0, 0), (-2, 0, 0), (-3, 0, 0), (-3, -1, 0), (-2, -1, 0)]

In [16]:
h=0
All = []
def calc_r (eps, f = 8):
 
    mean_c_l = []
    
    for i in range(4, f+1):
        seq = get_sequences(i)
        conformat = []

        start_conformation = [(0,0, 0)]
        get_all_saws(conformat, i ,start_conformation)

        Z = 0.0
        R_sum = 0.0
        
        E_all = 0 

        for l in range(len(conformat)):
            for j in range(len(seq)):
                #if (j<len(seq)-1):
                    #continue
                
                E = Ising_energy(conformat[l], seq[j])
      
                E_all=E_all+E
                #print(E, E_all)
                #print(np.exp(-(-eps*E-h*np.sum(seq[j]))))
                Z = Z + np.exp(-(-eps*E-h*np.sum(seq[j])))
                r = end_to_end_distance(conformat[l])
                R_sum = R_sum + np.exp(-(-eps*E-h*np.sum(seq[j])))*r
                if (i==5):
                    All.append(E)

        mean_c_l.append(R_sum/Z)
        #break
        print(i)
        print(R_sum,Z)
        print("R  = ", R_sum/Z)
        print(E_all, Z)
        print("E = ", E_all/Z )
        
        
    fn = 'Ising_3D/radiues1'+str(eps)+'.pickle'
    with open(fn, 'wb') as f:
        pickle.dump(mean_c_l, f)        
        
    

In [24]:
calc_r(0.9, 7)

4
28322.755186114435 7979.481590650919
R  =  3.549448026711223
0 7979.481590650919
E =  0.0
5
596701.2063622944 118988.47717734956
R  =  5.014781435289109
0 118988.47717734956
E =  0.0
6
11801844.911152782 1956006.6873665638
R  =  6.033642414097262
0 1956006.6873665638
E =  0.0
7
224721426.01532018 30399226.93610452
R  =  7.392340156796004
0 30399226.93610452
E =  0.0


In [ ]:
plt.hist(All, bins = 9)

In [ ]:
calc_r(0, 7)

In [36]:
calc_r(0.9, 7)

4
28322.755186114435 7979.481590650919
R  =  3.549448026711223
0 7979.481590650919
E =  0.0
5
596701.2063622944 118988.47717734956
R  =  5.014781435289109
0 118988.47717734956
E =  0.0
6
11801844.911152782 1956006.6873665638
R  =  6.033642414097262
0 1956006.6873665638
E =  0.0
7
224721426.01532018 30399226.93610452
R  =  7.392340156796004
0 30399226.93610452
E =  0.0


In [ ]:
calc_r(0.8, 7)

In [8]:
#no 3d yet 

def center_of_mass(conformation):
    """Функция для нахождения координат центра масс.
    Подразумевается, что все аминокислоты в молекуле одинаковой массы, а между ними ничего с массой нет.
    Принимает конформацию - список из координат, возвращает координаты в виде коортежа.
    """
    x=0
    y=0
    for i in range(len(conformation)):
        x=x+(conformation[i][0]-conformation[0][0])
        y=y+(conformation[i][1]-conformation[0][1])
        
    return (x/len(conformation), y/len(conformation)) 

def square_radius_of_gyration(conformation):
    """Функция возвращает радиус вращения для данной конформации """
    cent_mass=center_of_mass(conformation)
    summa=0
    for i in range(len(conformation)):
        dist = (conformation[i][0]-cent_mass[0],  conformation[i][1]-cent_mass[1])
        summa=summa+dist[0]*dist[0]+dist[1]*dist[1]
    return summa/len(conformation)
    

In [9]:
def calc_r_gyration (eps, f = 8):

    mean_c_l = []
    
    for i in range(4, f+1):
        seq = get_sequences(i)
        conformat = []

        start_conformation = [(0,0)]
        get_all_saws(conformat, i ,start_conformation)

        Z = 0.0
        R_sum = 0.0

        for l in range(len(conformat)):
            for j in range(len(seq)):
                E = Ising_energy(conformat[l], seq[j])
                Z = Z + np.exp(-eps*E)
                r = square_radius_of_gyration(conformat[l])
                R_sum = R_sum + np.exp(-eps*E)*r

        mean_c_l.append(R_sum/Z)
        
 
            
        print(i)
        print(R_sum,Z, R_sum/Z)
        
        
    fn = 'Ising/radiues1'+str(eps)+'_gyr.pickle'
    with open(fn, 'wb') as f:
        pickle.dump(mean_c_l, f)        
        

In [13]:
calc_r_gyration(0.5, 9)

4
681.7238821435777 858.742463331268 0.7938630162749927
5
6213.934097431552 5470.247184080317 1.135951244674196
6
53967.83337974882 36469.462495951164 1.479808850644298
7
433974.42794130195 230746.3218309893 1.8807425596112755
8
3430048.223825537 1507905.6345244353 2.2747101312525495
9
25876275.96334907 9492969.084872395 2.7258359036041147


In [10]:
calc_r_gyration(0.8, 9)

4
1201.5115397967031 1560.928903075966 0.7697413619729924
5
13393.665052724224 12196.1421903517 1.0981886602896347
6
148709.32843590336 106530.50121774133 1.3959319325077733
7
1478559.1895109913 836004.8171040502 1.7686012798739268
8
15060256.620164337 7210703.25405901 2.0885974764925597
9
142006079.74203935 57056848.58054875 2.4888524914159147


In [16]:
calc_r(0, 9)

4
2624.0 576.0
5
22528.0 3200.0
6
173824.0 18176.0
7
1255424.0 99840.0
8
8649728.0 556032.0
9
57589760.0 3028992.0


In [9]:
calc_r(0, 9)

4
2624.0 576.0
5
22528.0 3200.0
6
173824.0 18176.0
7
1255424.0 99840.0
8
8649728.0 556032.0
9
57589760.0 3028992.0


In [9]:
calc_r(0.5, 9)


4
2301314.7671979447 559102.5589982035
4.116086986476017
5
127225222.20031191 20022237.357214194
6.354196083609583
6
6429960937.67582 800525389.4827659
8.032176146006233
7
305342776295.6921 28781448383.5583
10.609013564102712
8
13994564463899.232 1134493693507.6902
12.335515432113215
9
620855601333435.1 41043134031759.05
15.12690529073679


In [10]:
calc_r(0.5, 9)

4
3795.209668892801 858.742463331268
5
37312.923012477615 5470.247184080317
6
331093.8458863125 36469.462495951164
7
2754353.1265523713 230746.3218309893
8
21914197.889178168 1507905.6345244353
9
168616103.63162214 9492969.084872395


In [18]:
calc_r(0.8, 9)


4
2667.1916731270217 619.1916731270205
5
23564.60015504843 3545.5333850161933
6
188630.015635034 21691.75723609176
7
1417002.4756924948 123819.65726853596
8
10206165.11825616 743246.7419526328
9
71123942.77211504 4234400.026969783


In [11]:
calc_r(0.8, 9)

4
6460.385930086536 1560.928903075966
5
77952.37717293733 12196.1421903517
6
860239.0706876665 106530.50121774133
7
8918268.76338377 836004.8171040502
8
89298294.74877487 7210703.25405901
9
865641491.5100735 57056848.58054875


In [19]:
calc_r(1.3, 9)

4
2748.277021481804 700.2770214818084
5
25510.648515563735 4194.216171854423
6
218945.16564905495 29551.65089116937
7
1764426.8843524102 178914.61306710588
8
13852065.182455745 1251987.7692973707
9
105179763.76126094 7710227.959430352


In [12]:
calc_r(1.3, 9)

4
22105.87517150113 6426.3420133735535
5
435316.2680159763 80080.32855429556
6
8335701.681888831 1418683.5275521083
7
147784967.43690336 18166659.024148706
8
2684658640.3004537 333091986.3716222
9
46704209216.32553 4587856888.732316
